In [13]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def g(x, x_0, theta, lamb):
    return 1 - (1 / (1 + np.exp(-theta * x))) + lamb * np.abs(x_0 - x)

x = np.linspace(-100, 100, 500)

def plot_g(x_0=0.0, theta=1.0, lamb=0.5):
    y = g(x, x_0, theta, lamb)
    plt.figure(figsize=(8, 4))
    plt.plot(x, y, lw=2)
    plt.title("g(x, x₀, θ, λ)")
    plt.xlabel("x")
    plt.ylabel("g(x)")
    plt.grid(True)
    plt.show()

interact(
    plot_g,
    x_0=FloatSlider(min=-20, max=20, step=0.1, value=0.0, description='x₀'),
    theta=FloatSlider(min=-20, max=20, step=0.1, value=1.0, description='θ'),
    lamb=FloatSlider(min=-0, max=10, step=0.1, value=0.5, description='λ')
)

def f(x, x_z, theta_1, theta_2, lamb):
    term1 = 1 - 1/(1 + np.exp(-theta_1 * x))
    term2 = 1 - 1/(1 + np.exp(-theta_2 * x))
    return term1 * term2 + lamb * np.abs(x_z - x)

x = np.linspace(-100, 100, 500)

def plot_f(x_z=0.0, theta_1=1.0, theta_2=1.0, lamb=0.5):
    y = f(x, x_z, theta_1, theta_2, lamb)
    plt.figure(figsize=(8, 4))
    plt.plot(x, y, lw=2)
    plt.title(r"$f(x, x_z, \theta _1, \theta _2, \lambda)$")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.grid(True)
    plt.show()

interact(
    plot_f,
    x_z=FloatSlider(min=-20, max=10, step=0.1, value=0.0, description=r'$x_z$'),
    theta_1=FloatSlider(min=-20, max=10, step=0.1, value=1.0, description=r'$\theta_1$'),
    theta_2=FloatSlider(min=-20, max=10, step=0.1, value=1.0, description=r'$\theta_2$'),
    lamb=FloatSlider(min=0, max=2, step=0.1, value=0.5, description=r'$\lambda$')
)


interactive(children=(FloatSlider(value=0.0, description='x₀', max=20.0, min=-20.0), FloatSlider(value=1.0, de…

interactive(children=(FloatSlider(value=0.0, description='$x_z$', max=10.0, min=-20.0), FloatSlider(value=1.0,…

<function __main__.plot_f(x_z=0.0, theta_1=1.0, theta_2=1.0, lamb=0.5)>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, HBox, VBox, Output
import IPython.display as display

# Your function
def g(x, x_0, theta, lamb):
    return 1 - (1 / (1 + np.exp(-theta * x))) + lamb * np.abs(x_0 - x)

# Define domain x
x = np.linspace(-10, 10, 500)

# Loss function: sum of g over x (arbitrary choice)
def loss(x_0, theta, lamb):
    return np.sum(g(x, x_0, theta, lamb))

# Numerical gradient of loss w.r.t. x_0 using finite differences
def grad_loss_x0(x_0, theta, lamb, eps=1e-5):
    return (loss(x_0 + eps, theta, lamb) - loss(x_0 - eps, theta, lamb)) / (2 * eps)

# Initial parameter values
x0_init = 0.0
theta_init = 1.0
lamb_init = 0.5

# Widgets
x0_slider = FloatSlider(min=-10, max=10, step=0.01, value=x0_init, description="x₀")
theta_slider = FloatSlider(min=-5, max=5, step=0.1, value=theta_init, description="θ")
lamb_slider = FloatSlider(min=-2, max=2, step=0.1, value=lamb_init, description="λ")

step_button = Button(description="Run 1 GD Step")
reset_button = Button(description="Reset")
out = Output()

# Variable to keep track of current x_0 during gradient descent
current_x0 = x0_init

def plot(x_0, theta, lamb):
    y = g(x, x_0, theta, lamb)
    plt.figure(figsize=(8, 4))
    plt.plot(x, y, label='g(x, x₀, θ, λ)')
    plt.axvline(x=x_0, color='red', linestyle='--', label=f"x₀ = {x_0:.3f}")
    plt.title("Function g(x, x₀, θ, λ) with current x₀")
    plt.xlabel("x")
    plt.ylabel("g(x)")
    plt.legend()
    plt.grid(True)
    plt.show()

def on_step_clicked(b):
    global current_x0
    lr = 0.1  # fixed learning rate for demo; you can add a slider if you want
    theta = theta_slider.value
    lamb = lamb_slider.value
    
    grad = grad_loss_x0(current_x0, theta, lamb)
    current_x0 = current_x0 - lr * grad
    # Update slider and plot
    x0_slider.value = current_x0
    
    with out:
        out.clear_output(wait=True)
        plot(current_x0, theta, lamb)

def on_reset_clicked(b):
    global current_x0
    current_x0 = x0_slider.value
    with out:
        out.clear_output(wait=True)
        plot(current_x0, theta_slider.value, lamb_slider.value)

# Connect button clicks to handlers
step_button.on_click(on_step_clicked)
reset_button.on_click(on_reset_clicked)

# When sliders change (except x0, which is updated by GD), update plot
def on_param_change(change):
    if change['name'] == 'value':
        with out:
            out.clear_output(wait=True)
            plot(current_x0, theta_slider.value, lamb_slider.value)

theta_slider.observe(on_param_change)
lamb_slider.observe(on_param_change)

# Initial plot
with out:
    plot(current_x0, theta_slider.value, lamb_slider.value)

# Layout
ui = VBox([x0_slider, theta_slider, lamb_slider, HBox([step_button, reset_button]), out])
display.display(ui)
